# Ridge Regression with Gradient Descent

In this notebook, we will implement ridge regression via gradient descent. We will:
* Convert an SFrame into a Numpy array
* Write a predict_output() function using Numpy
* Write a numpy function to compute the derivative of the regression weights with respect to a single feature
* Write gradient descent function to compute the regression weights given an initial weight vector, step size, tolerance, and L2 penalty

Make sure you have the latest version of Turi Create

In [11]:
import turicreate
import numpy as np
import matplotlib.pyplot as plt
import os
os.getcwd()

'/home/ian/venv/UWashington_MachineLearningSpecialization/Notebooks/GitHub'

# Load in House Sales Data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [12]:
sales = turicreate.SFrame(r'GitHub Data\home_data.sframe')
sales.head()

id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront
7129300520,2014-10-13 00:00:00+00:00,221900.0,3.0,1.0,1180.0,5650.0,1.0,0
6414100192,2014-12-09 00:00:00+00:00,538000.0,3.0,2.25,2570.0,7242.0,2.0,0
5631500400,2015-02-25 00:00:00+00:00,180000.0,2.0,1.0,770.0,10000.0,1.0,0
2487200875,2014-12-09 00:00:00+00:00,604000.0,4.0,3.0,1960.0,5000.0,1.0,0
1954400510,2015-02-18 00:00:00+00:00,510000.0,3.0,2.0,1680.0,8080.0,1.0,0
7237550310,2014-05-12 00:00:00+00:00,1225000.0,4.0,4.5,5420.0,101930.0,1.0,0
1321400060,2014-06-27 00:00:00+00:00,257500.0,3.0,2.25,1715.0,6819.0,2.0,0
2008000270,2015-01-15 00:00:00+00:00,291850.0,3.0,1.5,1060.0,9711.0,1.0,0
2414600126,2015-04-15 00:00:00+00:00,229500.0,3.0,1.0,1780.0,7470.0,1.0,0
3793500160,2015-03-12 00:00:00+00:00,323000.0,3.0,2.5,1890.0,6560.0,2.0,0


# Convert to Numpy Array

Although SFrames offer a number of benefits to users (especially when using Big Data and built-in Turi Create functions), in order to understand the details of the implementation of algorithms it's important to work with a library that allows for direct (and optimized) matrix operations. Numpy is a Python solution to work with matrices (or any multi-dimensional "array").

Recall that the predicted value given the weights and the features is just the dot product between the feature and weight vector. Similarly, if we put all of the features row-by-row in a matrix then the predicted value for *all* the observations can be computed by right multiplying the "feature matrix" by the "weight vector". 
 
Now we will write a function that will accept an SFrame, a list of feature names (['sqft_living', 'sqft_living_15']) and a target feature ('price') and will return two things:
* A numpy matrix whose columns are the desired features plus a constant column (this is how we create an intercept)
* A numpy array containing the values of the output

In [13]:
def get_numpy_data(data_sframe, features, output):
    data_sframe['constant'] = 1 # this is how you add a constant column to an SFrame
    
    # add the column 'constant' to the front of the features list so that we can extract it along with the others:
    features = ['constant'] + features # this is how you combine two lists
    
    # select the columns of data_SFrame given by the features list into the SFrame features_sframe (now including constant):
    features_sframe=data_sframe[features]
    
    # the following line will convert the features_SFrame into a numpy matrix:
    feature_matrix = features_sframe.to_numpy()
    
    # assign the column of data_sframe associated with the output to the SArray output_sarray
    output_sarray=data_sframe[output]

    # the following will convert the SArray into a numpy array by first converting it to a list
    output_array = output_sarray.to_numpy()
    return(feature_matrix, output_array)

We now define an additional function to compute the predictions for an entire matrix of features given the matrix and the weights:

In [14]:
def predict_output(feature_matrix, weights):
    # assume feature_matrix is a numpy matrix containing the features as columns and weights is a corresponding numpy array
    # create the predictions vector by using np.dot()
    predictions=np.dot(feature_matrix, weights)

    return(predictions)

# Computing the Derivative

We are now going to move to computing the derivative of the regression cost function. Recall that the cost function is the sum over the data points of the squared difference between an observed output and a predicted output, plus the L2 penalty term. Since the derivative of a sum is the sum of the derivatives, we can take the derivative of the first part (the RSS) as we did in the notebook for the unregularized case and add the derivative of the regularization part. 

**We will not regularize the constant.**  As a result, the derivative is just twice the sum of the errors (without the `2*l2_penalty*w[0]` term).

In [15]:
def feature_derivative_ridge(errors, feature, weight, l2_penalty, feature_is_constant):
    # If feature_is_constant is True, derivative is twice the dot product of errors and feature
    if feature_is_constant==True:
        derivative=2*np.dot(errors,feature)
    # Otherwise, derivative is twice the dot product plus 2*l2_penalty*weight
    else:
        derivative=2*np.dot(errors,feature)+2*l2_penalty*weight
    
    return derivative

# Gradient Descent

Now we will write a function that performs a gradient descent. The basic premise is simple. Given a starting point we update the current weights by moving in the negative gradient direction. Recall that the gradient is the direction of *increase* and therefore the negative gradient is the direction of *decrease* and we're trying to *minimize* a cost function. 

The amount by which we move in the negative gradient *direction*  is called the 'step size'. We stop when we are 'sufficiently close' to the optimum. Unlike in the *Linear Regression with Gradient Descent* notebook, this time we will set a **maximum number of iterations** and take gradient steps until we reach this maximum number. If no maximum number is supplied, the maximum should be set 100 by default. (Use default parameter values in Python.)

In [20]:
def ridge_regression_gradient_descent(feature_matrix, output, initial_weights, step_size, l2_penalty, max_iterations=100):
    print 'Starting gradient descent with l2_penalty = ' + str(l2_penalty)
    
    weights = np.array(initial_weights) # make sure it's a numpy array
    iteration = 0 # iteration counter
    print_frequency = 1  # for adjusting frequency of debugging output
    
    #while not reached maximum number of iterations:
    while iteration<max_iterations:
        iteration += 1  # increment iteration counter
        ### === code section for adjusting frequency of debugging output. ===
        #if iteration == 10:
         #   print_frequency = 10
        #if iteration == 100:
        #    print_frequency = 100
        #if iteration%print_frequency==0:
         #   print('Iteration = ' + str(iteration))
        ### === end code section ===
        
        # compute the predictions based on feature_matrix and weights using your predict_output() function
        predictions=predict_output(feature_matrix, weights)
        # compute the errors as predictions - output
        errors=predictions-output
        # from time to time, print the value of the cost function
        if iteration%print_frequency==10000:
            print 'Cost function = ', str(np.dot(errors,errors) + l2_penalty*(np.dot(weights,weights) - weights[0]**2))
        
        for i in xrange(len(weights)): # loop over each weight
            # Recall that feature_matrix[:,i] is the feature column associated with weights[i]
            # compute the derivative for weight[i].
            if i==0:
                derivative = feature_derivative_ridge(errors, feature_matrix[:,i], weights[i], l2_penalty, True)
            
            else:
                derivative = feature_derivative_ridge(errors, feature_matrix[:,i], weights[i], l2_penalty, False)
           
        # subtract the step size times the derivative from the current weight
            weights[i]=weights[i]-step_size*derivative
            
    print 'Done with gradient descent at iteration ', iteration
    print 'Learned weights = ', str(weights)
    return weights

The L2 penalty gets its name because it causes weights to have small L2 norms than otherwise. Let's see how large weights get penalized. Let us consider a simple model with 1 feature:

Let us split the dataset into training set and test set. Make sure to use `seed=0`:

In [23]:
model_features = ['sqft_living', 'sqft_living15'] # sqft_living15 is the average squarefeet for the nearest 15 neighbors. 
my_output = 'price'
initial_weights = np.array([-95000.0,0.0,0.0])
step_size = 1e-12
max_iterations = 50000
l2_penalty=0.1

In [24]:
(feature_matrix, output) = get_numpy_data(sales, model_features, my_output)

algorithm_weights = ridge_regression_gradient_descent(feature_matrix, output, 
                                                               initial_weights, step_size, 
                                                               l2_penalty, max_iterations)

Starting gradient descent with l2_penalty = 0.1
Done with gradient descent at iteration  50000
Learned weights =  [-9.50008536e+04  2.42140227e+02  6.63822836e+01]


In [25]:
model_ridge = turicreate.linear_regression.create(sales, 
                                                  target=my_output,
                                                  features=model_features,
                                                  verbose=False,
                                                  max_iterations=max_iterations,
                                                  l2_penalty=l2_penalty)
model_ridge.coefficients

name,index,value,stderr
(intercept),None,-98690.7716965,5573.28646657
sqft_living,None,242.648400999,3.02263052844
sqft_living15,None,67.3473827588,4.04983565927


As you can see, the coefficient weights on sqft_living and sqft_living15 estimated using Turi Create are very similar to the weights estimated by our algorithm!